# Single Scenario

In [1]:
import json  # needed to load in our scenario
import os  # to handle file paths across operating systems
from src.post_and_poll import get_api_results
API_KEY = 'DEMO_KEY'  # REPLACE WITH YOUR API KEY

In [2]:
# following is not necessary but silences warnings:
# InsecureRequestWarning: Unverified HTTPS request is being made to host 'developer.nrel.gov'. Adding certificate verification is strongly advised.
import urllib3
urllib3.disable_warnings()

`get_api_results` POST's your JSON to the API `job` endpoint, which provides a `run_uuid` if the input is valid, and then polls the `results` endpoint using the `run_uuid` until the results come back with a status other than `Optimizing...`.

`get_api_results` also saves the results to the `results_file`.

A log file is also created in the current working directory.

In [3]:
post = json.load(
    open(
        os.path.join("inputs", "Scenario_POST1.json"), 
    "r")
)

In [4]:
results = get_api_results(
    post=post,
    API_KEY=API_KEY,
    api_url="https://developer.nrel.gov/api/reopt/v1",
    results_file=os.path.join("outputs", "my_results.json")
)

main         INFO     Response OK from http://127.0.0.1:8000/v1/job/?api_key=DEMO_KEY.
main         INFO     Polling http://127.0.0.1:8000/v1/job/b2bd0eb4-654d-4366-8dbd-85fff00cdca9/results/?api_key=DEMO_KEY for results with interval of 5s...
main         INFO     Saved results to outputs/my_results.json


(Note the above logging shows a local host URL, which was used to run this notebook without requiring an API_KEY)

### Here are some results keys examples:

In [5]:
results.keys()

dict_keys(['outputs', 'inputs', 'messages'])

In [6]:
results["outputs"]["Scenario"]["status"]

'optimal'

In [7]:
for k in results["outputs"]["Scenario"]["Site"].keys():
    print(k)

year_one_emissions_lb_C02
year_one_emissions_bau_lb_C02
outdoor_air_temp_degF
renewable_electricity_energy_pct
Financial
LoadProfile
LoadProfileBoilerFuel
LoadProfileChillerThermal
ElectricTariff
FuelTariff
Storage
Generator
Wind
CHP
Boiler
ElectricChiller
AbsorptionChiller
HotTES
ColdTES
PV


In [8]:
for k in results["outputs"]["Scenario"]["Site"]["PV"].keys():
    print(k)

pv_name
size_kw
station_latitude
station_longitude
station_distance_km
average_yearly_energy_produced_kwh
average_yearly_energy_produced_bau_kwh
average_yearly_energy_exported_kwh
year_one_energy_produced_kwh
year_one_energy_produced_bau_kwh
year_one_power_production_series_kw
year_one_to_battery_series_kw
year_one_to_load_series_kw
year_one_to_grid_series_kw
year_one_curtailed_production_series_kw
existing_pv_om_cost_us_dollars
lcoe_us_dollars_per_kwh


# Multi-scenario
Multiple scenarios can be defined in a CSV file with rows for each scenrario and columns for the inputs. `all_api_inputs.csv` provides a template for all of the possible header values (input keys).

Let's take a look at the example `scenarios.csv`:

In [9]:
import pandas as pd  # only used to show the csv file

df = pd.read_csv(os.path.join("inputs", "scenarios.csv"))

df.head()

,site_number,description,latitude,longitude,PV|max_kw,urdb_label,load_file,Wind|max_kw
0,1,site3-No-PV,34.1404,-118.2764,0.0,5a3821035457a32645d2dd80,site3_8760_loads_kw.csv,0
1,2,site3-With-PV,34.1404,-118.2764,NaN,5a3821035457a32645d2dd80,site3_8760_loads_kw.csv,0


We have two Scenarios:
1. One with no PV nor Wind evaluated (by setting their `max_kw`s to zero)
2. One with PV and no Wind evaluated

Both Scenarios have the same location, the same electricity tariff (set via the `urdb_label`), and use the same custom load profile - which is passed in via another csv file.

In [10]:
"""
Here are some convenience definitions for using the Multi-scenario capabilities
"""
##############################################################################################################
inputs_path = os.path.join(".", 'inputs')
outputs_path = os.path.join(".", 'outputs')
output_template = os.path.join(outputs_path, 'results_template.csv')
output_file = os.path.join(outputs_path, 'results_summary.csv')
##############################################################################################################

Let's start by converting the `scenarios.csv` into a list of POST's that we can send to the API:

In [11]:
from src.multi_site_inputs_parser import multi_site_csv_parser

path_to_inputs = os.path.join(inputs_path, 'scenarios.csv')

list_of_posts = multi_site_csv_parser(
    path_to_inputs, 
    api_url='https://developer.nrel.gov/api/reopt/v1', 
    API_KEY=API_KEY
)

Now we can collect all of the results using the `get_api_results` function from the Single Scenario example:

In [12]:
responses = []

for post in list_of_posts:
    responses.append(
        get_api_results(
            post, 
            results_file=os.path.join(outputs_path, post['Scenario']['description'] + '.json'),
            api_url="https://developer.nrel.gov/api/reopt/v1", 
            API_KEY=API_KEY
        )
    )

main         INFO     Response OK from http://127.0.0.1:8000/v1/job/?api_key=DEMO_KEY.
main         INFO     Polling http://127.0.0.1:8000/v1/job/13aab880-0305-4fc9-9ce8-4047c96456a9/results/?api_key=DEMO_KEY for results with interval of 5s...
main         INFO     Saved results to ./outputs/site3-No-PV.json
main         INFO     Response OK from http://127.0.0.1:8000/v1/job/?api_key=DEMO_KEY.
main         INFO     Polling http://127.0.0.1:8000/v1/job/150de35c-febb-4a38-9ad0-ddca99d4873d/results/?api_key=DEMO_KEY for results with interval of 5s...
main         INFO     Saved results to ./outputs/site3-With-PV.json


Note that we used the `Scenario.description` to define the results file name. The `Scenario.description` was defined in the `scenarios.csv`.

### Summarizing multiple scenario results

There are two options for making a summary of multiple scenarios' resutls:
1. Write to a csv using a template with column headers for desired summary keys (scalar values only)
2. Write all inputs, outputs, and dispatch to an Excel spreadsheet


#### Option 1: Use a template CSV to collect certain results

In [13]:
from src.parse_api_responses_to_csv import parse_responses_to_csv_with_template

parse_responses_to_csv_with_template(
    csv_template=output_template, 
    responses=responses, 
    output_csv=output_file, 
    input_csv=path_to_inputs,
    n_custom_columns=2
)

#### Option 2: Write all results out to an Excel file

In [14]:
from src.parse_api_responses_to_excel import parse_api_responses_to_excel

parse_api_responses_to_excel(responses, spreadsheet='results_summary.xlsx')

/Users/nlaws/projects/reopt/REopt-API-Analysis/env/lib/python3.6/site-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
